# Data Modelling and Transformation

In [1]:
# import libraries
import os
import sys

In [2]:
from pathlib import Path
from importlib import reload
# add the project root to the path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
#import the module
from src.database.Connection import Database
import src.database.Connection
# reload the module to ensure we have the latest version
reload(src.database.Connection)

<module 'src.database.Connection' from '/home/chalasimon/Documents/10academy/week 7/challenge/Telegram-Medical-Insights/src/database/Connection.py'>

In [7]:
host = "localhost"
port = 5432
# get the database credentials from environment variables or use defaults
# you can set these in a .env file or directly in your environment
# for example, using dotenv package to load from .env file
from dotenv import load_dotenv
load_dotenv()

host=os.getenv("POSTGRES_HOST")
dbname=os.getenv("POSTGRES_DB")
user=os.getenv("POSTGRES_USER")
password=os.getenv("POSTGRES_PASSWORD")
port=os.getenv("POSTGRES_PORT")

# create a database connection
db = Database(host=host,database=dbname, user=user, password=password, port=port)
# connect to the database
con = db.connect()

Connection to the database established successfully.


In [8]:
# create table
create_telegram_table = """
create schema if not exists raw;

create table if not exists raw.telegram_messages (
  channel_title      text,
  channel_username   text not null,
  id                 bigint not null,
  text               text,
  date               timestamptz,
  views              integer,
  media_type         text,
  media_path         text,
  load_ts            timestamptz not null default now(),
  source_file        text not null,
  constraint telegram_messages_pk primary key (channel_username, id)
);
"""
# create the tables
db.create_table(create_telegram_table)

Table created successfully.
